# Compile with the following OPTIONS File:

*# Options for compiling*<br>
**SHELL = /bin/bash**

*# Choose the COMPILER*<br>
**COMPILER = ifort**

*# Choose the Linear Algebra libraries*<br>
**LINALG = MKL**

*# Do you use DEBUG or Optimization Mode? (DEBUG/OPT)*<br>
**MODE = OPT**

*# Turn on omp or mpi (enter TRUE .or. FALSE)?*<br>
**OPENMP = FALSE**<br>
**MPI = FALSE**

*# Choose the MACHINE file*<br>
**MACHINE = HKQAI**

*# Choose HORSFIELD or McWEDA*<br>
**THEORY = HORSFIELD**

*# Choose HARRIS, DOGS, or KS (Kohn-Sham)*<br>
*# Note that the KS code only does LDA for now as it uses McWEDA routines.*<br> 
**SCF = DOGS**

*# Choose the DIAGONALIZATION Method - GAMMAD (divide and conquer),*<br>
*# KPOINTS, or MDET (also GAMMA) - Default is GAMMA*<br>
**DIAGONALIZATION = GAMMAD**

*# Choose the XC functional*<br>
**XC = BLYP**


# Preparation

In [1]:
# link the executables
!rm fireball.x
!ln -s ~/thunder2022/fireball/src/fireball.x\

!rm fireball-ase.x
!ln -s ~/thunder2022/fireball/src/fireball-ase.x

In [2]:
# link the Fdata
!rm Fdata
!ln -s ~/thunder2022/datafiles/Fdata-Horsfield-0.10-9SN.Hs4.10-9DN.Cs4.35p4.80.Ns3.95p4.40.Os3.35p3.80 Fdata

In [3]:
from thunder_ase.fireball import Fireball
import numpy as np
import ase
import sys
from ase.build import molecule
from ase import units

# Socket Communication
from ase.calculators.socketio import SocketIOCalculator

# ase BFGS optimization
from ase.optimize import BFGS

# ase MD modules
from ase.md.verlet import VelocityVerlet as NVE
from ase.md.nvtberendsen import NVTBerendsen

## Example 1: Optimization (without socket communications)

In [4]:
# clean last run result
!./clean.com

rm: cannot remove 'WARNINGS': No such file or directory


In [5]:
# Read methane structure
atoms = ase.io.read('methane.xyz')

kwargs = {
          'iwriteout_charges': 1,
          }
fireball = Fireball(command='./fireball.x', Fdata_path='Fdata', **kwargs)

atoms.set_calculator(fireball)
dyn = BFGS(atoms, trajectory='opt.traj', logfile='opt.log')
dyn.run(fmax=0.05)  # run until maximum force is 0.05 eV/A

FIREBALL RUNTIME :      1.90 [sec]  
FIREBALL RUNTIME :      1.82 [sec]  
FIREBALL RUNTIME :      2.21 [sec]  


True

## Example 2: Optimization (with socket communications)

In [6]:
# clean last run result
!./clean.com

rm: cannot remove 'WARNINGS': No such file or directory


In [7]:
# Read methane structure
atoms = ase.io.read('methane.xyz')

# Sockets
host = 'thunder-ase'

kwargs = {
          'nstepi':1,
          'nstepf':100,
          'ipi': 1,
          'inet': 0,
          'host': 'thunder-ase',
          'iwriteout_charges': 1,
          }
fireball = Fireball(command='./fireball-ase.x', Fdata_path='Fdata', **kwargs)
fireball.prefix = ''
dyn = BFGS(atoms, trajectory='opt.traj', logfile='opt.log')

with SocketIOCalculator(fireball, log=sys.stdout,
                        unixsocket=host) as calc:
    atoms.set_calculator(calc)
    dyn.run(fmax=0.05)  # run until maximum force is 0.05 eV/A

Accepting clients on UNIX-socket /tmp/ipi_thunder-ase
Awaiting client
Accepted connection from 
Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver:   sendmsg 'POSDATA'
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 4 bytes of <class 'numpy.int32'>
Driver:   send 120 bytes of <class 'numpy.float64'>
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'HAVEDATA'
Driver:  sendrecv_force
Driver:   sendmsg 'GETFORCE'
Driver:   recvmsg 'FORCEREADY'
Driver:   recv 8 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 120 bytes of <class 'numpy.float64'>
Driver:   recv 72 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 1 bytes of <class 'numpy.int8'>
Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver

 ENDED 


## Example 2: NVE Ensemble

In [8]:
# clean last run result
!./clean.com

In [9]:
# Read methane structure
atoms = ase.io.read('methane.xyz')

# Sockets
host = 'thunder-ase'

kwargs = {
          'nstepi':1,
          'nstepf':1001,
          'ipi': 1,
          'inet': 0,
          'host': 'thunder-ase',
          'iwriteout_charges': 1,
          }
fireball = Fireball(command='./fireball-ase.x', Fdata_path='Fdata', **kwargs)
fireball.prefix = ''
dyn = NVE(atoms, timestep=0.25 * units.fs, trajectory='md.nve.traj', logfile='md.nve.log')

with SocketIOCalculator(fireball, log=sys.stdout,
                        unixsocket=host) as calc:
    atoms.set_calculator(calc)
    atoms.rattle(stdev=0.1)
    dyn.run(1000)  # run until maximum force is 0.05 eV/A

Accepting clients on UNIX-socket /tmp/ipi_thunder-ase
Awaiting client
Accepted connection from 
Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver:   sendmsg 'POSDATA'
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 4 bytes of <class 'numpy.int32'>
Driver:   send 120 bytes of <class 'numpy.float64'>
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'HAVEDATA'
Driver:  sendrecv_force
Driver:   sendmsg 'GETFORCE'
Driver:   recvmsg 'FORCEREADY'
Driver:   recv 8 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 120 bytes of <class 'numpy.float64'>
Driver:   recv 72 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 1 bytes of <class 'numpy.int8'>
Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver

Driver:   send 4 bytes of <class 'numpy.int32'>
Driver:   send 120 bytes of <class 'numpy.float64'>
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'HAVEDATA'
Driver:  sendrecv_force
Driver:   sendmsg 'GETFORCE'
Driver:   recvmsg 'FORCEREADY'
Driver:   recv 8 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 120 bytes of <class 'numpy.float64'>
Driver:   recv 72 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 1 bytes of <class 'numpy.int8'>
Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver:   sendmsg 'POSDATA'
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 4 bytes of <class 'numpy.int32'>
Driver:   send 120 bytes of <class 'numpy.float64'>
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'HAVEDATA'
Driver:  sendrecv_force
Driver:   send

Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 120 bytes of <class 'numpy.float64'>
Driver:   recv 72 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 1 bytes of <class 'numpy.int8'>
Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver:   sendmsg 'POSDATA'
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 4 bytes of <class 'numpy.int32'>
Driver:   send 120 bytes of <class 'numpy.float64'>
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'HAVEDATA'
Driver:  sendrecv_force
Driver:   sendmsg 'GETFORCE'
Driver:   recvmsg 'FORCEREADY'
Driver:   recv 8 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 120 bytes of <class 'numpy.float64'>
Driver:   recv 72 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   r

Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver:   sendmsg 'POSDATA'
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 4 bytes of <class 'numpy.int32'>
Driver:   send 120 bytes of <class 'numpy.float64'>
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'HAVEDATA'
Driver:  sendrecv_force
Driver:   sendmsg 'GETFORCE'
Driver:   recvmsg 'FORCEREADY'
Driver:   recv 8 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 120 bytes of <class 'numpy.float64'>
Driver:   recv 72 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 1 bytes of <class 'numpy.int8'>
Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver:   sendmsg 'POSDATA'
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 4 bytes of <class 

Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'HAVEDATA'
Driver:  sendrecv_force
Driver:   sendmsg 'GETFORCE'
Driver:   recvmsg 'FORCEREADY'
Driver:   recv 8 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 120 bytes of <class 'numpy.float64'>
Driver:   recv 72 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 1 bytes of <class 'numpy.int8'>
Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver:   sendmsg 'POSDATA'
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 4 bytes of <class 'numpy.int32'>
Driver:   send 120 bytes of <class 'numpy.float64'>
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'HAVEDATA'
Driver:  sendrecv_force
Driver:   sendmsg 'GETFORCE'
Driver:   recvmsg 'FORCEREADY'
Driver:   recv 8 bytes of <class 'numpy.float64'>
Driver:   recv 4 byt

...

Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver:   sendmsg 'POSDATA'
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 4 bytes of <class 'numpy.int32'>
Driver:   send 120 bytes of <class 'numpy.float64'>
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'HAVEDATA'
Driver:  sendrecv_force
Driver:   sendmsg 'GETFORCE'
Driver:   recvmsg 'FORCEREADY'
Driver:   recv 8 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 120 bytes of <class 'numpy.float64'>
Driver:   recv 72 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 1 bytes of <class 'numpy.int8'>
Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver:   sendmsg 'POSDATA'
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 4 bytes of <class 

Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'HAVEDATA'
Driver:  sendrecv_force
Driver:   sendmsg 'GETFORCE'
Driver:   recvmsg 'FORCEREADY'
Driver:   recv 8 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 120 bytes of <class 'numpy.float64'>
Driver:   recv 72 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 1 bytes of <class 'numpy.int8'>
Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver:   sendmsg 'POSDATA'
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 4 bytes of <class 'numpy.int32'>
Driver:   send 120 bytes of <class 'numpy.float64'>
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'HAVEDATA'
Driver:  sendrecv_force
Driver:   sendmsg 'GETFORCE'
Driver:   recvmsg 'FORCEREADY'
Driver:   recv 8 bytes of <class 'numpy.float64'>
Driver:   recv 4 byt

Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 1 bytes of <class 'numpy.int8'>
Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver:   sendmsg 'POSDATA'
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 4 bytes of <class 'numpy.int32'>
Driver:   send 120 bytes of <class 'numpy.float64'>
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'HAVEDATA'
Driver:  sendrecv_force
Driver:   sendmsg 'GETFORCE'
Driver:   recvmsg 'FORCEREADY'
Driver:   recv 8 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 120 bytes of <class 'numpy.float64'>
Driver:   recv 72 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 1 bytes of <class 'numpy.int8'>
Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver:

Driver:   send 4 bytes of <class 'numpy.int32'>
Driver:   send 120 bytes of <class 'numpy.float64'>
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'HAVEDATA'
Driver:  sendrecv_force
Driver:   sendmsg 'GETFORCE'
Driver:   recvmsg 'FORCEREADY'
Driver:   recv 8 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 120 bytes of <class 'numpy.float64'>
Driver:   recv 72 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 1 bytes of <class 'numpy.int8'>
Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver:   sendmsg 'POSDATA'
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 4 bytes of <class 'numpy.int32'>
Driver:   send 120 bytes of <class 'numpy.float64'>
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'HAVEDATA'
Driver:  sendrecv_force
Driver:   send

 ENDED 


## Example 3: NVT Ensemble

In [10]:
# clean last run result
!./clean.com

rm: cannot remove 'WARNINGS': No such file or directory


In [11]:
# Read methane structure
atoms = ase.io.read('methane.xyz')

# Sockets
host = 'thunder-ase'

kwargs = {
          'nstepi':1,
          'nstepf':1001,
          'ipi': 1,
          'inet': 0,
          'host': 'thunder-ase',
          'iwriteout_charges': 1,
          }
fireball = Fireball(command='./fireball-ase.x', Fdata_path='Fdata', **kwargs)
fireball.prefix = ''
dyn = NVTBerendsen(atoms, timestep=0.25 * units.fs, temperature = 300, taut = 0.5*1000*units.fs,
                   trajectory='md.nvt.traj', logfile='md.nvt.log')

with SocketIOCalculator(fireball, log=sys.stdout,
                        unixsocket=host) as calc:
    atoms.set_calculator(calc)
    atoms.rattle(stdev=0.1)
    dyn.run(1000)  # run until maximum force is 0.05 eV/A

Accepting clients on UNIX-socket /tmp/ipi_thunder-ase
Awaiting client
Accepted connection from 
Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver:   sendmsg 'POSDATA'
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 4 bytes of <class 'numpy.int32'>
Driver:   send 120 bytes of <class 'numpy.float64'>
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'HAVEDATA'
Driver:  sendrecv_force
Driver:   sendmsg 'GETFORCE'
Driver:   recvmsg 'FORCEREADY'
Driver:   recv 8 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 120 bytes of <class 'numpy.float64'>
Driver:   recv 72 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 1 bytes of <class 'numpy.int8'>
Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver

/users/lewis/anaconda3/lib/python3.10/site-packages/ase/md/nvtberendsen.py:95: RuntimeWarning: divide by zero encountered in double_scalars
  (self.temperature / old_temperature - 1.0) *


Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver:   sendmsg 'POSDATA'
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 4 bytes of <class 'numpy.int32'>
Driver:   send 120 bytes of <class 'numpy.float64'>
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'HAVEDATA'
Driver:  sendrecv_force
Driver:   sendmsg 'GETFORCE'
Driver:   recvmsg 'FORCEREADY'
Driver:   recv 8 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 120 bytes of <class 'numpy.float64'>
Driver:   recv 72 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 1 bytes of <class 'numpy.int8'>
Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver:   sendmsg 'POSDATA'
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 72 bytes

Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'HAVEDATA'
Driver:  sendrecv_force
Driver:   sendmsg 'GETFORCE'
Driver:   recvmsg 'FORCEREADY'
Driver:   recv 8 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 120 bytes of <class 'numpy.float64'>
Driver:   recv 72 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 1 bytes of <class 'numpy.int8'>
Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver:   sendmsg 'POSDATA'
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 4 bytes of <class 'numpy.int32'>
Driver:   send 120 bytes of <class 'numpy.float64'>
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'HAVEDATA'
Driver:  sendrecv_force
Driver:   sendmsg 'GETFORCE'
Driver:   recvmsg 'FORCEREADY'
Driver:   recv 8 bytes of <class 'numpy.float64'>
Driv

Driver:   recv 72 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 1 bytes of <class 'numpy.int8'>
Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver:   sendmsg 'POSDATA'
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 4 bytes of <class 'numpy.int32'>
Driver:   send 120 bytes of <class 'numpy.float64'>
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'HAVEDATA'
Driver:  sendrecv_force
Driver:   sendmsg 'GETFORCE'
Driver:   recvmsg 'FORCEREADY'
Driver:   recv 8 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 120 bytes of <class 'numpy.float64'>
Driver:   recv 72 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 1 bytes of <class 'numpy.int8'>
Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Dri

...

Driver:   recv 72 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 1 bytes of <class 'numpy.int8'>
Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver:   sendmsg 'POSDATA'
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 4 bytes of <class 'numpy.int32'>
Driver:   send 120 bytes of <class 'numpy.float64'>
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'HAVEDATA'
Driver:  sendrecv_force
Driver:   sendmsg 'GETFORCE'
Driver:   recvmsg 'FORCEREADY'
Driver:   recv 8 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 120 bytes of <class 'numpy.float64'>
Driver:   recv 72 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 1 bytes of <class 'numpy.int8'>
Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Dri

Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 4 bytes of <class 'numpy.int32'>
Driver:   send 120 bytes of <class 'numpy.float64'>
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'HAVEDATA'
Driver:  sendrecv_force
Driver:   sendmsg 'GETFORCE'
Driver:   recvmsg 'FORCEREADY'
Driver:   recv 8 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 120 bytes of <class 'numpy.float64'>
Driver:   recv 72 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 1 bytes of <class 'numpy.int8'>
Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver:   sendmsg 'POSDATA'
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 4 bytes of <class 'numpy.int32'>
Driver:   send 120 bytes of <class 'numpy.float64'>
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvms

Driver:   recv 8 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 120 bytes of <class 'numpy.float64'>
Driver:   recv 72 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 1 bytes of <class 'numpy.int8'>
Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver:   sendmsg 'POSDATA'
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 4 bytes of <class 'numpy.int32'>
Driver:   send 120 bytes of <class 'numpy.float64'>
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'HAVEDATA'
Driver:  sendrecv_force
Driver:   sendmsg 'GETFORCE'
Driver:   recvmsg 'FORCEREADY'
Driver:   recv 8 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 120 bytes of <class 'numpy.float64'>
Driver:   recv 72 bytes of <class 'numpy.float64'>
Driver:  

Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver:   sendmsg 'POSDATA'
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 4 bytes of <class 'numpy.int32'>
Driver:   send 120 bytes of <class 'numpy.float64'>
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'HAVEDATA'
Driver:  sendrecv_force
Driver:   sendmsg 'GETFORCE'
Driver:   recvmsg 'FORCEREADY'
Driver:   recv 8 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 120 bytes of <class 'numpy.float64'>
Driver:   recv 72 bytes of <class 'numpy.float64'>
Driver:   recv 4 bytes of <class 'numpy.int32'>
Driver:   recv 1 bytes of <class 'numpy.int8'>
Driver: calculate
Driver:  status
Driver:   sendmsg 'STATUS'
Driver:   recvmsg 'READY'
Driver:  sendposdata
Driver:   sendmsg 'POSDATA'
Driver:   send 72 bytes of <class 'numpy.float64'>
Driver:   send 72 bytes

ConnectionResetError: [Errno 104] Connection reset by peer